In [1]:
from prep import MeterDataSet
import pandas as pd
import numpy as np
from webapp.utils.azure_utils import KeyVault, DataLake

In [2]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)
file_system = "energyhub"

In [3]:
meter = "electricity"
metadata_cols = ['building_id', 'site_id','sq_meter']
weather_cols = ['site_id', 'timestamp', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precipitation_depth_1_hr',
    'precipitation_depth_6_hr', 'sea_level_pressure', 'wind_direction',
    'wind_speed']

In [4]:
electricity = MeterDataSet(meter, metadata_cols, weather_cols)

In [5]:
electricity.merge()

In [7]:
df = electricity.df

In [18]:
weather = electricity.weather

In [23]:
sites = weather.site_id.unique().compute()
cols = weather.columns[2:]

In [24]:
site = sites[0]
col = cols[0]

In [25]:
weather = weather.compute()
dfs = []
for site in sites:
    df = weather.loc[weather.site_id == site, :]
    df = df.set_index("timestamp")
    for col in cols:
        if col == "cloud_coverage":
            df.loc[:, col] = df.loc[:, col].fillna(method = "bfill")
        else:
            df.loc[:, col] = df.loc[:, col].interpolate(method = "linear").fillna(method = "bfill")
    df = df.reset_index()
    dfs.append(df)

weather = pd.concat(dfs)

In [27]:
weather.isna().sum()*100/len(weather)

timestamp                    0.000000
site_id                      0.000000
air_temperature              0.000000
cloud_coverage              10.296347
dew_temperature              0.000000
precipitation_depth_1_hr    26.437799
precipitation_depth_6_hr     5.284359
sea_level_pressure           5.284359
wind_direction               0.000000
wind_speed                   0.000000
dtype: float64